In [1]:
from IPython.display import display, HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

In [1]:
spark

In [7]:
dfStat = spark.read.format('csv').load('D1.csv')

In [8]:
dfStat.show()

+-------+-----+--------+------+-----------+
|    _c0|  _c1|     _c2|   _c3|        _c4|
+-------+-----+--------+------+-----------+
|   date|delay|distance|origin|destination|
|1011245|    6|     602|   ABE|        ATL|
|1020600|   -8|     369|   ABE|        DTW|
|1021245|   -2|     602|   ABE|        ATL|
|1020605|   -4|     602|   ABE|        ATL|
|1031245|   -4|     602|   ABE|        ATL|
|1030605|    0|     602|   ABE|        ATL|
|1041243|   10|     602|   ABE|        ATL|
|1040605|   28|     602|   ABE|        ATL|
|1051245|   88|     602|   ABE|        ATL|
|1050605|    9|     602|   ABE|        ATL|
|1061215|   -6|     602|   ABE|        ATL|
|1061725|   69|     602|   ABE|        ATL|
|1061230|    0|     369|   ABE|        DTW|
|1060625|   -3|     602|   ABE|        ATL|
|1070600|    0|     369|   ABE|        DTW|
|1071725|    0|     602|   ABE|        ATL|
|1071230|    0|     369|   ABE|        DTW|
|1070625|    0|     602|   ABE|        ATL|
|1071219|    0|     569|   ABE| 

In [9]:
dfStat.isStreaming

False

In [10]:
df = spark.readStream.format('socket')\
.option('host','localhost')\
.option('port',12345)\
.load()

24/06/01 11:14:47 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.


In [11]:
df.isStreaming

True

In [12]:
df.printSchema()

root
 |-- value: string (nullable = true)



In [5]:
from pyspark.sql.functions import *
import pyspark.sql.functions as F

In [14]:
dfsplit = df.withColumn('Splitteddata',split('value',' '))

In [15]:
dfsplit.printSchema()

root
 |-- value: string (nullable = true)
 |-- Splitteddata: array (nullable = true)
 |    |-- element: string (containsNull = false)



In [17]:
dfsize = dfsplit.withColumn('Size',size('Splitteddata'))

In [18]:
dfsize.printSchema()

root
 |-- value: string (nullable = true)
 |-- Splitteddata: array (nullable = true)
 |    |-- element: string (containsNull = false)
 |-- Size: integer (nullable = false)



In [22]:
writer = dfsize.writeStream.format('console')\
.outputMode('append')\
.option('truncate','false')\
.trigger(processingTime='2 seconds')

In [23]:
writer

In [24]:
q = writer.start()

24/06/01 11:33:56 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-9dc42b38-fd60-48c7-8e0a-9b0b04c045a4. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/06/01 11:33:56 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+-----+------------+----+
|value|Splitteddata|Size|
+-----+------------+----+
+-----+------------+----+

-------------------------------------------
Batch: 1
-------------------------------------------
+-----------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----+
|value                                                                                                                                                |Splitteddata                                                                                                                                                                  |Size|
+------------------------------------

-------------------------------------------
Batch: 3
-------------------------------------------
+-----+------------+----+
|value|Splitteddata|Size|
+-----+------------+----+
|     |[]          |1   |
+-----+------------+----+

-------------------------------------------
Batch: 4
-------------------------------------------
+-------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------+----+
|value                                                                                                  |Splitteddata                                                                                                               |Size|
+-------------------------------------------------------------------------------------------------------+----------------------------------------------------------------------------------------------------

In [25]:
q.stop()

In [26]:
dfsplit.printSchema()

root
 |-- value: string (nullable = true)
 |-- Splitteddata: array (nullable = true)
 |    |-- element: string (containsNull = false)



In [27]:
dfwords = dfsplit.select(explode('Splitteddata').alias('Words'))

In [28]:
dfwords.printSchema()

root
 |-- Words: string (nullable = false)



In [29]:
dfcount = dfwords.groupBy('Words').count()

In [30]:
dfcount.printSchema()

root
 |-- Words: string (nullable = false)
 |-- count: long (nullable = false)



In [33]:
writer = dfcount.writeStream.format('console')\
.outputMode('complete')\
.option('truncate','false')\
.trigger(processingTime='2 seconds')

In [34]:
q = writer.start()

24/06/01 11:41:23 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-8da7dd64-031b-4779-8687-802593dcfaff. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/06/01 11:41:23 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+-----+-----+
|Words|count|
+-----+-----+
+-----+-----+



24/06/01 11:41:40 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 2000 milliseconds, but spent 16548 milliseconds


-------------------------------------------
Batch: 1
-------------------------------------------
+-----+-----+
|Words|count|
+-----+-----+
|you  |1    |
|how  |1    |
|Hi   |1    |
|are  |1    |
|all  |1    |
|?    |1    |
+-----+-----+



24/06/01 11:42:58 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 2000 milliseconds, but spent 10660 milliseconds
24/06/01 11:43:45 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 2000 milliseconds, but spent 9964 milliseconds


-------------------------------------------
Batch: 2
-------------------------------------------
+-----+-----+
|Words|count|
+-----+-----+
|you  |1    |
|how  |2    |
|Hi   |2    |
|are  |1    |
|all  |1    |
|?    |1    |
+-----+-----+



-------------------------------------------
Batch: 3
-------------------------------------------
+---------------+-----+
|Words          |count|
+---------------+-----+
|those          |1    |
|some           |1    |
|Cannot         |1    |
|operation      |1    |
|joins          |1    |
|Instead,       |1    |
|sense          |1    |
|count()        |1    |
|returns        |1    |
|aggregations   |2    |
|starting       |1    |
|not            |7    |
|Distinct       |1    |
|will           |2    |
|functionalities|1    |
|by             |1    |
|Rather,        |1    |
|you            |1    |
|outer          |1    |
|Deduplication  |1    |
+---------------+-----+
only showing top 20 rows



24/06/01 11:45:09 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 2000 milliseconds, but spent 7765 milliseconds


In [35]:
q.stop()

In [37]:
writer = dfcount.writeStream.format('console')\
.outputMode('complete')\
.option('truncate','false')\
.option('numrows',100)\
.trigger(processingTime='20 seconds')

In [38]:
q = writer.start() 

24/06/01 11:47:58 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-be04ee91-f227-456f-9804-99cf91b84f33. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/06/01 11:47:58 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+-----+-----+
|Words|count|
+-----+-----+
+-----+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+----------------------------+-----+
|Words                       |count|
+----------------------------+-----+
|those                       |1    |
|some                        |1    |
|Cannot                      |1    |
|operation                   |1    |
|joins                       |1    |
|Instead,                    |1    |
|sense                       |1    |
|count()                     |1    |
|returns                     |1    |
|aggregations                |2    |
|starting                    |1    |
|not                         |7    |
|Distinct                    |1    |
|will                        |2    |
|functionalities             |1    |
|by                          |1    |
|Rather,                     |1    |
|you                         |1    |
|outer                       |1    |
|Deduplication               |1    |
|In                          |1    |
|there         

-------------------------------------------
Batch: 2
-------------------------------------------
+----------------------------+-----+
|Words                       |count|
+----------------------------+-----+
|Cannot                      |1    |
|those                       |1    |
|some                        |1    |
|operation                   |1    |
|Instead,                    |1    |
|joins                       |1    |
|sense                       |1    |
|count()                     |1    |
|returns                     |1    |
|aggregations                |2    |
|starting                    |1    |
|not                         |7    |
|Distinct                    |1    |
|will                        |2    |
|functionalities             |1    |
|by                          |1    |
|Rather,                     |1    |
|you                         |1    |
|outer                       |1    |
|Deduplication               |1    |
|In                          |1    |
|there         

In [40]:
q.stop()

In [42]:
q= writer.start()

24/06/01 11:53:13 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-6d6c318d-9e12-4510-9a8a-eb2366a9ce64. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/06/01 11:53:13 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+-----+-----+
|Words|count|
+-----+-----+
+-----+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+-----------------------------------+-----+
|Words                              |count|
+-----------------------------------+-----+
|few                                |3    |
|those                              |1    |
|some                               |1    |
|recognize                          |1    |
|input                              |9    |
|operation                          |2    |
|import                             |9    |
|developers,                        |1    |
|received                           |2    |
|column                             |2    |
|map                                |1    |
|latencies</b>                      |1    |
|achieve                            |1    |
|applications)                      |1    |
|place,                             |1    |
|If                                 |7    |
|API                                |3    |
|you.                  

In [43]:
q.stop()

In [44]:
q = writer.start()

24/06/01 12:04:14 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-ca9ff29a-eb77-4fbf-8e53-8b6da93e32d2. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/06/01 12:04:14 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+-----+-----+
|Words|count|
+-----+-----+
+-----+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+---------------------------------------+-----+
|Words                                  |count|
+---------------------------------------+-----+
|import                                 |1    |
|=                                      |6    |
|lines.select(split(col("value"),       |1    |
|"\\s").alias("word"))                  |1    |
|words.groupBy("word").count()          |1    |
|")).alias("word"))                     |1    |
|counts                                 |1    |
|#words                                 |1    |
|streamingQuery.stop()                  |2    |
|*                                      |1    |
|.option("port",                        |1    |
|words                                  |1    |
|(spark                                 |1    |
|.load())                               |1    |
|lines                                  |1    |
|.outputMode("complete")               

-------------------------------------------
Batch: 2
-------------------------------------------
+---------------------------------------+-----+
|Words                                  |count|
+---------------------------------------+-----+
|import                                 |1    |
|=                                      |6    |
|lines.select(split(col("value"),       |1    |
|"\\s").alias("word"))                  |1    |
|words.groupBy("word").count()          |1    |
|")).alias("word"))                     |1    |
|counts                                 |1    |
|you                                    |1    |
|#words                                 |1    |
|streamingQuery.stop()                  |2    |
|*                                      |1    |
|how                                    |1    |
|.option("port",                        |1    |
|words                                  |1    |
|(spark                                 |1    |
|.load())                              

In [45]:
q.stop()

In [1]:
s = 'date STRING, delay INTEGER, distance INTEGER, origin STRING, destination STRING'

In [2]:
df = spark.readStream.format('csv')\
.option('header','true')\
.schema(s)\
.load('MyInputStream')

In [3]:
df = spark.readStream.format('parquet')\
.schema(s)\
.load('MyInputStream')

In [5]:
df.isStreaming

True

In [6]:
df.printSchema()

root
 |-- date: string (nullable = true)
 |-- delay: integer (nullable = true)
 |-- distance: integer (nullable = true)
 |-- origin: string (nullable = true)
 |-- destination: string (nullable = true)



In [7]:
df2 = df.groupBy('date').agg(avg('delay').alias('AvgDelay'))

In [8]:
df2.isStreaming

True

In [53]:
df2.printSchema()

root
 |-- date: string (nullable = true)
 |-- AvgDelay: double (nullable = true)



In [17]:
writer = df2.writeStream.format('console')\
.outputMode('complete')\
.option('truncate','false')\
.option('numrows',100)\
.trigger(processingTime='20 seconds')

In [55]:
q=writer.start()

24/06/01 12:42:27 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-a3c6889d-0400-4c1a-aa11-3591e32b89e6. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/06/01 12:42:27 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+-------+--------+
|date   |AvgDelay|
+-------+--------+
|1011245|6.0     |
|1020605|-4.0    |
|1061230|0.0     |
|1021245|-2.0    |
|1061215|-6.0    |
|1030605|0.0     |
|1020600|-8.0    |
|1071230|0.0     |
|1041243|10.0    |
|1051245|88.0    |
|1040605|28.0    |
|1060625|-3.0    |
|1070600|0.0     |
|1031245|-4.0    |
|1061725|69.0    |
|1050605|9.0     |
|1071725|0.0     |
|1070625|0.0     |
|1071219|0.0     |
+-------+--------+



-------------------------------------------
Batch: 1
-------------------------------------------
+-------+--------+
|date   |AvgDelay|
+-------+--------+
|1011245|6.0     |
|1020605|-4.0    |
|1121215|-5.0    |
|1140625|-5.0    |
|1061230|0.0     |
|1021245|-2.0    |
|1061215|-6.0    |
|1121725|-1.0    |
|1030605|0.0     |
|1131230|-13.0   |
|1020600|-8.0    |
|1151219|0.0     |
|1150625|0.0     |
|1131215|14.0    |
|1131725|-6.0    |
|1071230|0.0     |
|1041243|10.0    |
|1131219|-8.0    |
|1051245|88.0    |
|1141230|-8.0    |
|1040605|28.0    |
|1151725|-6.0    |
|1130625|29.0    |
|1060625|-3.0    |
|1070600|0.0     |
|1151230|0.0     |
|1130600|-7.0    |
|1031245|-4.0    |
|1061725|69.0    |
|1050605|9.0     |
|1141725|-9.0    |
|1140600|-9.0    |
|1071725|0.0     |
|1      |2.0     |
|1070625|0.0     |
|1071219|0.0     |
|1150607|0.0     |
|1150600|0.0     |
|1141219|-10.0   |
+-------+--------+



In [10]:
q=writer.start()

24/06/01 12:48:00 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-2da1f4a7-c449-4b4d-9f43-f2fa347233b6. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/06/01 12:48:00 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+-------+--------+
|date   |AvgDelay|
+-------+--------+
|1011245|6.0     |
|1020605|-4.0    |
|1121215|-5.0    |
|1140625|-5.0    |
|1061230|0.0     |
|1021245|-2.0    |
|1061215|-6.0    |
|1121725|-1.0    |
|1030605|0.0     |
|1131230|-13.0   |
|1020600|-8.0    |
|1151219|0.0     |
|1131215|14.0    |
|1150625|0.0     |
|1131725|-6.0    |
|1071230|0.0     |
|1041243|10.0    |
|1131219|-8.0    |
|1051245|88.0    |
|1141230|-8.0    |
|1040605|28.0    |
|1151725|-6.0    |
|1130625|29.0    |
|1060625|-3.0    |
|1070600|0.0     |
|1151230|0.0     |
|1130600|-7.0    |
|1031245|-4.0    |
|1061725|69.0    |
|1050605|9.0     |
|1141725|-9.0    |
|1140600|-9.0    |
|1071725|0.0     |
|1      |2.0     |
|1070625|0.0     |
|1071219|0.0     |
|1150607|0.0     |
|1150600|0.0     |
|1141219|-10.0   |
+-------+--------+



24/06/01 12:48:27 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 20000 milliseconds, but spent 25779 milliseconds


In [20]:
q.stop()

In [19]:
q=writer.start()

24/06/01 12:51:36 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-88490d25-4386-492e-aeeb-ea203443081d. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/06/01 12:51:36 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+-------+--------+
|date   |AvgDelay|
+-------+--------+
|1110600|-9.0    |
|1110625|-4.0    |
|1090600|151.0   |
|1100600|-5.0    |
|1080607|5.0     |
|1081219|54.0    |
|1111215|127.0   |
|1091725|0.0     |
|1091230|-4.0    |
|1090625|8.0     |
|1101725|7.0     |
|1091215|43.0    |
|1100625|52.0    |
|1080625|1.0     |
|1091219|83.0    |
|1101219|0.0     |
|1101215|-5.0    |
|1101230|-8.0    |
|1081230|33.0    |
|1080600|0.0     |
+-------+--------+



-------------------------------------------
Batch: 1
-------------------------------------------
+-------+--------+
|date   |AvgDelay|
+-------+--------+
|1011245|6.0     |
|1020605|-4.0    |
|1110600|-9.0    |
|1110625|-4.0    |
|1061230|0.0     |
|1090600|151.0   |
|1100600|-5.0    |
|1021245|-2.0    |
|1061215|-6.0    |
|1080607|5.0     |
|1030605|0.0     |
|1020600|-8.0    |
|1081219|54.0    |
|1111215|127.0   |
|1071230|0.0     |
|1041243|10.0    |
|1051245|88.0    |
|1091725|0.0     |
|1091230|-4.0    |
|1040605|28.0    |
|1090625|8.0     |
|1101725|7.0     |
|1060625|-3.0    |
|1070600|0.0     |
|1091215|43.0    |
|1031245|-4.0    |
|1061725|69.0    |
|1050605|9.0     |
|1100625|52.0    |
|1080625|1.0     |
|1091219|83.0    |
|1071725|0.0     |
|1101219|0.0     |
|1101215|-5.0    |
|1101230|-8.0    |
|1070625|0.0     |
|1071219|0.0     |
|1081230|33.0    |
|1080600|0.0     |
+-------+--------+



24/06/01 12:53:00 ERROR Executor: Exception in task 0.0 in stage 8.0 (TID 807)
java.lang.RuntimeException: file:/home/hatem/PySpark/Intake 44/L5_StructuredStreaming/MyInputStream/D1.csv is not a Parquet file. Expected magic number at tail, but found [79, 82, 68, 10]
	at org.apache.parquet.hadoop.ParquetFileReader.readFooter(ParquetFileReader.java:565)
	at org.apache.parquet.hadoop.ParquetFileReader.<init>(ParquetFileReader.java:799)
	at org.apache.parquet.hadoop.ParquetFileReader.open(ParquetFileReader.java:666)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFooterReader.readFooter(ParquetFooterReader.java:85)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFooterReader.readFooter(ParquetFooterReader.java:71)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFooterReader.readFooter(ParquetFooterReader.java:66)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat.$anonfun$buildReaderWithPartitionValues$2(ParquetFileFormat.scal

In [28]:
writer = df.writeStream.format('parquet')\
.outputMode('append')\
.option('path','Outstream')\
.option('checkpointLocation','chkpointMans')\
.trigger(processingTime='2 seconds')

In [29]:
q = writer.start()

24/06/01 13:00:39 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [30]:
q.stop()

In [8]:
writer = df2.writeStream.format('memory')\
.outputMode('complete')\
.queryName('myQuery')\
.trigger(processingTime='2 seconds')

In [9]:
q = writer.start()

24/06/01 13:06:56 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-3497c455-dd1d-4b43-abff-f9f58b9795a5. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/06/01 13:06:56 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/06/01 13:07:20 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 2000 milliseconds, but spent 23791 milliseconds


In [10]:
df_q = spark.sql('select * from myQuery')

In [11]:
df_q.isStreaming

False

In [13]:
df_q.show(100)

+-------+--------+
|   date|AvgDelay|
+-------+--------+
|1011245|     6.0|
|1020605|    -4.0|
|1110600|    -9.0|
|1110625|    -4.0|
|1061230|     0.0|
|1090600|   151.0|
|1100600|    -5.0|
|1021245|    -2.0|
|1061215|    -6.0|
|1080607|     5.0|
|1030605|     0.0|
|1020600|    -8.0|
|1081219|    54.0|
|1111215|   127.0|
|1071230|     0.0|
|1041243|    10.0|
|1051245|    88.0|
|1091725|     0.0|
|1091230|    -4.0|
|1040605|    28.0|
|1090625|     8.0|
|1101725|     7.0|
|1060625|    -3.0|
|1070600|     0.0|
|1091215|    43.0|
|1031245|    -4.0|
|1061725|    69.0|
|1050605|     9.0|
|1100625|    52.0|
|1080625|     1.0|
|1091219|    83.0|
|1071725|     0.0|
|1101219|     0.0|
|1101215|    -5.0|
|1101230|    -8.0|
|1070625|     0.0|
|1071219|     0.0|
|1081230|    33.0|
|1080600|     0.0|
+-------+--------+



24/06/01 13:08:17 WARN ProcessingTimeExecutor: Current batch is falling behind. The trigger interval is 2000 milliseconds, but spent 15165 milliseconds


In [15]:
df_q = spark.sql('select * from myQuery')
df_q.show(100)

+-------+--------+
|   date|AvgDelay|
+-------+--------+
|1011245|     6.0|
|1020605|    -4.0|
|1121215|    -5.0|
|1140625|    -5.0|
|1110600|    -9.0|
|1110625|    -4.0|
|1061230|     0.0|
|1090600|   151.0|
|1100600|    -5.0|
|1021245|    -2.0|
|1061215|    -6.0|
|1080607|     5.0|
|1121725|    -1.0|
|1030605|     0.0|
|1131230|   -13.0|
|1020600|    -8.0|
|1151219|     0.0|
|1081219|    54.0|
|1150625|     0.0|
|1131215|    14.0|
|1111215|   127.0|
|1131725|    -6.0|
|1071230|     0.0|
|1041243|    10.0|
|1131219|    -8.0|
|1051245|    88.0|
|1091725|     0.0|
|1091230|    -4.0|
|1141230|    -8.0|
|1040605|    28.0|
|1151725|    -6.0|
|1090625|     8.0|
|1130625|    29.0|
|1101725|     7.0|
|1060625|    -3.0|
|1070600|     0.0|
|1091215|    43.0|
|1151230|     0.0|
|1130600|    -7.0|
|1031245|    -4.0|
|1061725|    69.0|
|1050605|     9.0|
|1100625|    52.0|
|1141725|    -9.0|
|1080625|     1.0|
|1140600|    -9.0|
|1091219|    83.0|
|1071725|     0.0|
|1101219|     0.0|
|1101215|   

In [16]:
q.stop()